In [1]:
#!pip install chesslab

In [2]:
import numpy as np
from chesslab.utils import load_pkl
from chesslab.training_tf import fitting
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:

lr = 0.1
epochs=20
batch_size = 128
test_percent=0.1

path = 'D:/database/ccrl/'
name_data='ccrl_states_full.pkl'
name_labels='ccrl_results_full.pkl'
save_name='./tmp/tf_weights-relu-full.1'

optim = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [4]:
download=False
if download:
    from chesslab.utils import download_7z
    path='./'
    file_id = '1MFHFz_rxNziYSeN-9ruwnRiYskd0_9ss'
    download_7z(file_id,path)

In [5]:
encoding_3={
    '.':np.array([0,0,0,0,0,0,0,0,0,0,0,0],dtype=np.float32),
    'p':np.array([1,0,0,0,0,0,0,0,0,0,0,0],dtype=np.float32),
    'P':np.array([0,0,0,0,0,0,0,0,0,0,0,1],dtype=np.float32),
    'b':np.array([0,1,0,0,0,0,0,0,0,0,0,0],dtype=np.float32),
    'B':np.array([0,0,0,0,0,0,0,0,0,0,1,0],dtype=np.float32),
    'n':np.array([0,0,1,0,0,0,0,0,0,0,0,0],dtype=np.float32),
    'N':np.array([0,0,0,0,0,0,0,0,0,1,0,0],dtype=np.float32),
    'r':np.array([0,0,0,1,0,0,0,0,0,0,0,0],dtype=np.float32),
    'R':np.array([0,0,0,0,0,0,0,0,1,0,0,0],dtype=np.float32),
    'q':np.array([0,0,0,0,1,0,0,0,0,0,0,0],dtype=np.float32),
    'Q':np.array([0,0,0,0,0,0,0,1,0,0,0,0],dtype=np.float32),
    'k':np.array([0,0,0,0,0,1,0,0,0,0,0,0],dtype=np.float32),
    'K':np.array([0,0,0,0,0,0,1,0,0,0,0,0],dtype=np.float32)
}


In [6]:
class Model_3():

    def __init__(self,
                   n_classes=2):
        initializer = tf.keras.initializers.GlorotNormal()
        self.hw=[]
        self.hb=[]

        self.hw.append( tf.Variable(initializer(shape=(7,7,12,32),dtype=np.float32),name="hl1weigths",dtype="float32") )
        self.hb.append( tf.Variable(np.zeros(32,dtype=np.float32),name="hl1bias",dtype="float32") )
        #8x8x32
        self.hw.append( tf.Variable(initializer(shape=(5,5,32,64),dtype=np.float32),name="hl2weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(64,dtype=np.float32),name="hl2bias",dtype="float32"))
        #8x8x64
        self.hw.append( tf.Variable(initializer(shape=(3,3,64,128),dtype=np.float32),name="hl3weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(128,dtype=np.float32),name="hl3bias",dtype="float32"))
        #8x8x128
        self.hw.append( tf.Variable(initializer(shape=(8*8*128,256),dtype=np.float32),name="hl4weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(256,dtype=np.float32),name="hl4bias",dtype="float32"))

        self.hw.append( tf.Variable(initializer(shape=(256, n_classes),dtype=np.float32),name="outweigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(n_classes,dtype=np.float32),name="outbias",dtype="float32"))

        self.trainable_variables = []
        for i in range(len(self.hw)):
            self.trainable_variables.append(self.hw[i])    
            self.trainable_variables.append(self.hb[i])

    def __call__(self,x): 

        out = tf.cast(x, tf.float32)
        out = tf.reshape(out, shape=[-1, 8, 8, 12])

        layer=0
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*32
        layer+=1
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*64
        layer+=1  
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*128
        layer+=1
        out =  tf.reshape(out,[-1, 8*8*128])
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)

        layer+=1
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])

        return out

In [7]:
np.random.seed(0)
tf.random.set_seed(0)

x_data = load_pkl(path+name_data)
y_data = load_pkl(path+name_labels)[:,1] #Nota: pasa de onehot a logits

print(x_data.shape)
print(y_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = test_percent, random_state = 0, shuffle = True)

del x_data
del y_data

(17164805, 64)
(17164805,)


In [8]:

model = Model_3()
encoding=encoding_3


In [9]:
fitting(epochs=epochs,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model=model,
        optimizer=optim,
        batch_size=batch_size,
        lr=lr,
        loss_fn=loss_fn,
        save_name=save_name,
        encoding=encoding)

2021-10-29 10:56:17


Epoch: 01/20 | time: 740s = 12.3m | train loss: 0.4913 | train acc: 0.7512 | test loss: 0.4383 | test acc: 0.7872

Epoch: 02/20 | time: 731s = 12.2m | train loss: 0.4070 | train acc: 0.8054 | test loss: 0.3818 | test acc: 0.8200

Epoch: 03/20 | time: 767s = 12.8m | train loss: 0.3525 | train acc: 0.8362 | test loss: 0.3628 | test acc: 0.8351

Epoch: 04/20 | time: 735s = 12.2m | train loss: 0.3142 | train acc: 0.8568 | test loss: 0.5235 | test acc: 0.8014

...

Epoch: 16/20 | time: 613s = 10.2m | train loss: 0.2218 | train acc: 0.9041 | test loss: 0.3022 | test acc: 0.8799

Epoch: 17/20 | time: 621s = 10.4m | train loss: 0.2204 | train acc: 0.9048 | test loss: 0.2987 | test acc: 0.8811

Epoch: 18/20 | time: 616s = 10.3m | train loss: 0.2193 | train acc: 0.9055 | test loss: 0.2981 | test acc: 0.8811

Epoch: 19/20 | time: 627s = 10.4m | train loss: 0.2184 | train acc: 0.9059 | test loss: 0.4602 | test acc: 0.8511

Epoch: 20/20 | time: 626s = 10.4m | train loss: 0.2175 | train acc: 0.9064 | test loss: 0.3102 | test acc: 0.8801

In [10]:
fitting(epochs=10,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model= model, 
        load_name='tmp/tf_weights-relu-full.1.20.h5',
        save_name=save_name,)

2021-10-29 14:33:19


Epoch: 21/30 | time: 626s = 10.4m | train loss: 0.2167 | train acc: 0.9068 | test loss: 0.3063 | test acc: 0.8794

Epoch: 22/30 | time: 629s = 10.5m | train loss: 0.2159 | train acc: 0.9072 | test loss: 0.3007 | test acc: 0.8810

Epoch: 23/30 | time: 621s = 10.4m | train loss: 0.2157 | train acc: 0.9075 | test loss: 0.3032 | test acc: 0.8815

Epoch: 24/30 | time: 625s = 10.4m | train loss: 0.2153 | train acc: 0.9078 | test loss: 0.6018 | test acc: 0.8282

...

Epoch: 26/30 | time: 627s = 10.5m | train loss: 0.2143 | train acc: 0.9083 | test loss: 0.3125 | test acc: 0.8795

Epoch: 27/30 | time: 625s = 10.4m | train loss: 0.2138 | train acc: 0.9086 | test loss: 0.3152 | test acc: 0.8790

Epoch: 28/30 | time: 618s = 10.3m | train loss: 0.2137 | train acc: 0.9087 | test loss: 0.3096 | test acc: 0.8790

Epoch: 29/30 | time: 621s = 10.4m | train loss: 0.2134 | train acc: 0.9089 | test loss: 0.3553 | test acc: 0.8753

Epoch: 30/30 | time: 624s = 10.4m | train loss: 0.2133 | train acc: 0.9091 | test loss: 0.3085 | test acc: 0.8814